# Introduction to Advanced Architectures

Topics covered in this module

- Evolution of covolutional neural networks (1998 - 2014)
- ImageNet Dataset
- Network in Network: 1x1 Convolution
- InceptionNet
- ResNet 
- DenseNet
- Neural Architecture Search

## History: LeNet-5 (1998)

- **LeNet-5 [LeCun et al.]** was designed for digit recognition to be used by banks in 1998. It is a 7-layer CNN.

<img src="img/lenet.png" width=1000>


[[LeCun et al. 1998] Gradient Based Learning Applied to Document Recognition](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf)


## History: ImageNet


- Since 2010 [ImageNet Large Scale Visual Recognition Challenge (ILSVRC)](https://www.image-net.org/challenges/LSVRC/index.php) has been holding an annual challenge to evaluates algorithms computer vision tasks. 

- [ImageNet](https://en.wikipedia.org/wiki/ImageNet) is a database of more than 14 million hand-annotated images. 


<img src="img/imagenet.png" width=500>


## History: Deeper Networks

- Until 2014, most of the CNNs' advances were made through making the networks deeper and larger. It required lots of GPU-days to train a model and a lot of engineering effort went through it to train these networks. 

- **AlexNet [Krizhevsky et al. 2012]** had 5 convolution layers and 4 fully connected layers with a total of 60M parameters. It was trained on 2 GPUs (hence the split in the network). It had filters of sizes ranging from 11 to 3.

<img src="img/alexnet.png" width=1000>


[[Krizhevsky et al. 2012]ImageNet Classification with Deep Convolution Neural Networks](https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf)


## History: Deeper Networks

- **VGG-16 Net [Simonyan et al. 2014]** simplified AlexNet by using lots of 3x3 filters and 2x2 pooling layers only. There were 16 layers in the network with a total of 138M parameters

<img src="img/vgg.png" width=1000>

[[Simonyan et al. 2014] Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

## Advances 

- Researchers have come up with novel ways to improve the task accuracy

- In this unit, we will look at those elements that have led to further advances in the CNNs

- Note that this is not an exhaustive list

- These core ideas are the foundations for most current state-of-the-art architectures


## Network in Network: 1x1 convolution

- Each filter is defined by the number of input channels $C_{in}$ and the kernel size $f$ (assume standard values of padding, stride, and dilation). Thus, each filter has $C_{in} \times f \times f$  number of parameters


- For each filter, if the input is of size $n \times n$ then the output size is $ (n - f + 1) \times (n - f + 1)$. We call the output from each filter as a *feature map*. 


- For $f > 1$, there is a decrease in the receptive field with each convolution layer

- For $f = 1$, the receptive field remains the same. These are 1x1 convolutions. 
- **MLPConv** layer described by Lin et al. is a stack of 1x1 convolution layers akin to a multi-layer feed forward network.
- Thus, for a single layered MLPConv layer, if there are $C_{out}$ number of channels required in the output, there will be $C_{out}$ such number of 1x1 convolutions, i.e., $C_{in} \times 1 \times 1 \times C_{out}$ (plus $C_{out}$ bias parameters) number of parameters will be required

- It looks very much like a traditional MLP with input dimension $C_{in}$ and output dimension $C_{out}$, however, it is slided over the input feature map (hence the term convolution).

<figure>
    <img src="img/mlpconv.png" width=1000>
</figure>


[[Lin et al. 2014] Network in Network](https://arxiv.org/pdf/1312.4400.pdf)

## Network in Network


- Lin et al. calls **MLPConv** layer followed by *global average pooling* a **Network In Network** structure.

<img src="img/nin.png" width=1000>

- Lin et al. discusses the following benefits of using these
    - By keeping the receptive field unchanged, this structure enhances "model discriminability for local patches within the receptive field"
    - It allows "cascaded cross-channel poooling", thereby allowing "complex and learnable interactions of cross channel information"
    - In transitioning from convolution layers to traditional MLP layers, one can use as many 1x1 convolution as the number of categories followed by *global average pooling*. Compared to vectorizing the last feature maps to be fed to a fully-connected layer, this structure restricts the number of parameters, thereby reducing the chances of overfitting. 
- 1x1 Convolution has also been used to reduce the number of operations in a network (see next slides)

[[Lin et al. 2014] Network in Network](https://arxiv.org/pdf/1312.4400.pdf)

## Inception module

- There are several options available for filter sizes - $1 \times 1$, $3 \times 3$, $5 \times 5$, max-pooling, etc. 
- Instead of running a big hyperparameter search, the inception module uses them all
- The naïve version (Figure 2a) is computationally more expensive than the one with 1x1 convolutions (Figure 1b)
- Besides reducing the computaitonal complexity in Figure 1b, 1x1 convolutions also enable dimension reduction such that *"even low dimensional embeddings might contain a lot of information about a relatively large image patch"*
- Output of all convolution layers are concatenated together

<img src="img/inception.png" width=1000>

[[Szegedy et al. 2014] Going deeper with convolutions](https://arxiv.org/pdf/1409.4842.pdf)

## Inception module

- Computational cost (naïve version (Figure 2a))
    - Consider the output of size $n_{out} \times n_{out} \times C_{out}$
    - Each of the element in this output requires $f \times f \times C_{in}$ number of mulitplication operations ($f > 1$)
    - Thus, a total of $(n_{out} \times n_{out} \times C_{out}) \times (f \times f \times C_{in})$ operations are required,
    
    
   $$A =  (n_{out} \times n_{out} \times C_{out}) \times (f \times f \times C_{in}) $$
    
- Computational cost (with 1x1 convolutions Figure 1b)
    - 1x1 convolution reduces the input volume from $C_{in}$ to $C_{out}^{'}$ before applying the filter of size $f > 1$
    - Thus, intermediate output is of size $n_{in} \times n_{in} \times C_{int}$, where each element of this output requires $1 \times 1 \times C_{in}$ multiplication operations
    - Final output is the convolution over the intermediate output with the same number of input and output channels
    - Thus, if the filter of size $f > 1$ is used, $f \times f \times C_{int}$ mulitplication operations are used to output $n_{out} \times n_{out} \times C_{out}$ number of elements
    - Total number of operations are therefore, 
    
    $$B = \Big[(n_{in} \times n_{in} \times C_{int}) \times (1 \times 1 \times C_{in}) \Big] + \Big[(n_{out} \times n_{out} \times C_{out}) \times (f \times f \times C_{int})\Big]$$

- Comparing $A$ and $B$ and assuming $n_{in} \approx n_{out}$ we have,

$$\frac{B}{A} = \frac{C_{int}}{C_{out}}\times\frac{1}{f^2} + \frac{C_{int}}{C_{in}}$$

- With $f = 3$, $C_{int} = 16$, $C_{out} = 32$, $C_{in} = 192$, we get $B = 0.14A$, i.e., 86% reduction in the number of operations

- Number of addition operations are similar to multiplication operations


[[Szegedy et al. 2014] Going deeper with convolutions](https://arxiv.org/pdf/1409.4842.pdf)

## Inception Network / GoogLeNet

- It is composed of several stacked inception module
- This network has several softmax outputs which are used in the same way as the final output, i.e., as a classifier. These softmax layers stabilizes training.

<img src="img/3600px-Googlenet.png" width=1000>


[Image source](https://developer.ridgerun.com/wiki/index.php?title=R2Inference/Introduction)

## Residual Block


- **Skip Connections**: Makes the input to layer $l+1$ also available to the layer $l+2$
    - Enables the input to be available deeper in the network
    - Makes it easier for the network to learn an identity function if the need be (see next slide)

<table>
    <tr>
        <th style="text-align:center">plain network</th>
        <th style="text-align:center">Residual Block / Skip connection</th>
    </tr>
    <tr>
        <td><img src="img/plain-net.png" width="500px"><br></td>
        <td><img src="img/skip-conn.png" width="500px"><br></td>
    </tr>
</table>

## Residual Block


- Consider *information flow from layer $l$ to $l+2$* in a **plain network**, i.e., without skip connections
    * Let $f_l$ represent the affine transformation defined by $f_l(x) = W^{l}x + b^{l}$
    * Let $g$ represent the ReLU activation
    * The transformation from the layer $l$ to $l+2$ is then 
    
    $$ a^{l} \xrightarrow{g \ \circ\ f_{l+1}} a^{l+1} \xrightarrow{g \ \circ \ f_{l+2}} a^{l+2}$$
    
    * Mathematically, 

$$ z^{l+1} = W^{l+1}a^{l} + b^{l+1} \qquad a^{l+1} = g(z^{l+1}) \qquad z^{l+2} = W^{l+2}a^{l+1} + b^{l+2} \qquad a^{l+2} = g(z^{l+2}) $$



- Compare it to the same information flow in a **residual block**, i.e., with skip connections

    $$a^{l+2} = g(z^{l+2} + a^{l})$$

    * activation at layer $l$ can go deeper into the network
    * Note: the number of learnable parameters remain the same
    * skip connections changes the way input is processed

[[He et al. 2015] Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

## Residual Block

- If the network prefers to zero-out all the weights in layer, it can easily do so by learning an identity function

$$ a^{l+2} = ReLU(z^{l+2} + a^{l}) \implies a^{l+2} = ReLU(W^{l+2}a^{l+1} + b^{l+2} + a^{l})$$

- the network can easily force $W^{l+2}$ and $b^{l+2}$ to $0$ to learn an identity function$^1$,  i.e, $a^{l+2} = a^l$ 
- in a plain-network an identity function, i.e.,  $W^{l+2} = I$ and $b^{l+2} = 0$ might be difficult for the network to find
- empirically, one can add many such blocks without hindering the optimization procedure too much
- "plain-network" have a harder time to optimize with increasing number of layers

<img src="img/resnet-layer.png" width=500>


$^1$ Note: $a^{l} > 0$ because it is an output of ReLU 


[Image source](https://medium.com/machine-learning-bites/deeplearning-series-convolutional-neural-networks-a9c2f2ee1524)

## ResNet Architecture

ResNet architecture is composed of several of such residual blocks as shown below

**Note**: these skip connections are over convolution layers, therefore, padding, stride, and dilation are chosen such that the size remains same. Similar adjustments are made for the transition between convolution and pooling layers.

<img src="img/resnet.jpeg" width=1000>

[[He et al. 2015] Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

## ResNet 


<img src="img/resnet-loss.png" width=1000>


[[Li et al. 2018] Visualizing the Loss Landscape of Neural Nets](https://arxiv.org/pdf/1712.09913.pdf)

## Pre-activtaions: Improved ResNet 

- He et al. 2016 did several experiments to analyze the impact of different components in a residual unit. 


- Mathematically, a residual unit is expressed as 

$$y_l = h(x_l) + \mathcal{F}(x_l, W_l) \qquad x_{l+1} = f(y_l),$$

where $h$, $f$, and $\mathcal{F}$ are transformation functions.

- In the original Resnet, $h$ is an identity mapping, i.e., $h(x) = x$, and $f$ is ReLU activation. 


- He et al. found that shallow layer $l$ (early in the network) recieve clean gradients from deeper layers $L$ when $f$ and $h$ are an identity mapping. It can be seen by unrolling the above recursion from layer $l$ to $L$ through residual units

$$x_{L} = x_{l} + \sum \limits_{i=0}^{L-1} \mathcal{F}(x_i, W_i) \qquad \frac{\Delta\mathcal{E}}{\Delta x_l} = \frac{\Delta\mathcal{E}}{\Delta x_L}\frac{\Delta x_L}{\Delta x_l} = \frac{\Delta\mathcal{E}}{\Delta x_L}\Big(1 + \sum \limits_{i=0}^{L-1} \frac{\mathcal{F}(x_i, W_i)}{\Delta x_l}\Big)$$

- He et al. conducted two sets of experiments
    - (A) Design of $h$
    - (B) Design of $f$
    
  
- First, they looked at the impact of different forms of $h$ on the training. They concluded that $h(x) = x$, i.e., identity mapping always leads to an optimal choice. 

[[He et al. 2016] Identity mappings in Deep Residual Networks](https://arxiv.org/pdf/1603.05027.pdf)

## Pre-activtaions: Improved ResNet 

- Second, they experimented with various forms of $f$ to design an identity mapping. Assuming that $h$ is an identity mapping, the second activation $y_{l+1}$ will look as follows - 

$$x_{l+1} = f(y_{l}) \qquad y_{l+1} = f(y_l) + \mathcal{F}(f(y_l), W_l) \qquad x_{l+2} = f(y_{l+1}),$$

He et al. chose the following formulation to make the first term in  $y_{l+1}$ as an identity mapping 

$$y_{l+1} = y_l + \mathcal{F}(\hat{f}(y_l), W_l),$$

Thus, the problem reduced down to the design of $\hat{f}$ while enabling the similar equations as wanted with identity mappings.

- He et al. concluded that pre-activation $\hat{f} = BN \cdot ReLU \cdot W$ performs much better than post-activation (original residual unit with $f = ReLU$)


<table>
    <tr>
        <th style="text-align:center">Original unit</th>
        <th style="text-align:center">Pre-activation unit</th>
    </tr>
    <tr>
        <td><img src="img/og-resnet.png" width=250> <br></td>
        <td><img src="img/pre-activation.png" width="250px"><br></td>
    </tr>
</table>

The red activation unit in the picture above is the composition of $BN \cdot ReLU$


[[He et al. 2016] Identity mappings in Deep Residual Networks](https://arxiv.org/pdf/1603.05027.pdf)

## DenseNet: DenseBlocks

- Problems with the previous approaches
    - There are **too many parameters** making the network prone to overfitting
    - Each layer learns a representation based *only* on the previous layers' output. Thus, the **network need to be deeper to have superior representational power**. 
    - Deeper networks are **harder to train**: ResNets overcome this through skip-connections, however, ResNets *add* output from the previous layer "which may impede the information flow in the network".
    - If a deeper layer need earlier layers as an input to **learn more complex features**, the network will need to learn to copy earlier layers. Therefore, DenseNet uses all of the previous layers as an input to overcome this shortcoming 
 
 
- DenseBlock: Each block consists of several convolutional layers
    - **Improved information flow with dense connectivity**: Concatenates previous layers' output instead of adding them like in ResNets. Denoting $H_l$ as the non-linear transformation of the input, $l^{th}$ layer's input is the concatenation of all the previous layers' activation ($[]$ denotes the concatenation operator)
$$a_l = H_l([a_0, a_1, a_2, ..., a_{l-1}])$$
    - **Composite function**: $H_l$ is a composite function comprising of Batch-Normalization (BN), ReLU, and convolution layers
    
    $$H_l  = Conv^{3 \times 3, k} \circ ReLU \circ BN \circ Conv^{1 \times 1, 4k} \circ ReLU \circ BN, $$
    
    here $Conv^{f \times f,  k}$ is the convolution layer represents $k$ filters of size $f$. Note, the use of 1x1 convolution is inspired by the bottleneck idea of InceptionNet.
    
    - **Growth Rate**: Each layer outputs $k$ feature maps. Thus, the output of one dense block has $k_0 + k * (l-1)$ feature maps, where $k_0$ is the feature maps in the input to the dense block. Thus, $k$ is the growth rate. 
    

<img src="img/dense-block.png" width=500>


[[Huang et al. 2016]  Densely Connected Convolutional Networks](https://arxiv.org/abs/1608.06993)

## DenseNet


- The network is comprised of dense blocks with **transition layers** in between
- Denoting the transition layer after dense block $i$ as $T_i$, and the number of output feature maps of the $i^{th}$ dense block as $C_{out}^i$. $T_i$, as used by the authors, is 
    $$T_i = AveragePooling(K=2, Stride=1) \circ  Conv^{1x1, \theta \times C^i_{out}} \circ BN, $$
    
    here $\theta$ is the **compression factor** to compress the number of feature maps after each dense blocks.

<img src="img/densenet.png" width=1000>


- In this way, 
    - Network uses **fewer parameters** as there is no need to learn redundant feature maps
    - Densenet layers are **narrow** because they add only a small set of new feature-maps
    - Each layer has direct access to the gradients from the loss function, thereby making it **easier to train the network**


[[Huang et al. 2016]  Densely Connected Convolutional Networks](https://arxiv.org/abs/1608.06993)

## WaveNet: Dilated Causal Convolution

- [Oord et al. 2016] proposed WaveNet as a generative model for raw speech. It forms the basis of modern text-to-speech systems. 

- Generative model of a raw speech requires modeling a joint probability of observing the waveform $\{x_i\}_{i=1}^{T}$

$$p(x_1, x_2, ..., x_{T}) = \prod_{t=1}^{T}p(x_t \ | \ x_1, x_2, ..., x_{t-1}),$$

therefore, we are interested in parameterizing the neural network to predict $p(x_t \ | \ x_1, x_2, ..., x_{t-1})$.

- For a text-to-speech systems, this probability needs to be conditioned on the text $c$, thus, we model a conditional probability as

$$p(x_1, x_2, ..., x_{T}\ |\ c) = \prod_{t=1}^{T}p(x_t \ | \ x_1, x_2, ..., x_{t-1}, c)$$


[[Oord et al. 2016] WaveNet: A Generative Model For Raw Audio](https://arxiv.org/pdf/1609.03499.pdf)

## WaveNet: Dilated Causal Convolution

- **Causal Convolution**: To prevent the model from peaking ahead into the future, we want input for the prediction of $x_t$ to consists of values prior to $t$, i.e., $\{x_i\}_{i=1}^{t-1}$. 

    - This input can be further restricted by kernel size (receptive field) to include only a subset of the past values. 
    - Several such layers can be stacked on top of each other to produce a stacked causal convolution network

<img src="img/causal_conv.png" width=1000>


[[Oord et al. 2016] WaveNet: A Generative Model For Raw Audio](https://arxiv.org/pdf/1609.03499.pdf)

## WaveNet: Dilated Causal Convolution

- **Dilated Convolution** or **Convolution with holes**: A convolution applied on the input by skipping values at regular certain steps. It amounts to increasing the receptive field of the convolution. 

<img src="img/dilated_conv.png" width=1000>



[[Oord et al. 2016] WaveNet: A Generative Model For Raw Audio](https://arxiv.org/pdf/1609.03499.pdf)

## Separable Convolutions

- Advances in CNNs were dependent on making architectures deeper and computational heavy


- This prevents these CNNs in being used on devices that do not have a lot of compute, e.g., smartphones


- Separable convolutions were proposed to decompose the kernels and reduce the computational complexity of these networks


- Depthwise Separable Convolutions (DSC) have been successful in reducing the computational complexity of the architectures while achieving a similar performance as larger architectures. 
    - Applied in XceptionNet ([Francois Chollet 2017](https://openaccess.thecvf.com/content_cvpr_2017/papers/Chollet_Xception_Deep_Learning_CVPR_2017_paper.pdf)), MobileNets ([Howard et al.](https://arxiv.org/abs/1704.04861)), and many more ...

- Following separable convolutions have been studied in literature - 

    - **Spatially Separable Convolution**
    
    - **Depthwise Separable Convolution**


[[Francois Chollet 2017] Deep learning with Depthwise Separable Convolution](https://openaccess.thecvf.com/content_cvpr_2017/papers/Chollet_Xception_Deep_Learning_CVPR_2017_paper.pdf)

[[Howard et al. 2017] MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/abs/1704.04861)

## Spatially Separable Convolution

- **Spatially Separable Convolution**: 
    - Kernels are decomposed as a matrix multiplication of 2 vectors. 
    - E.g., A $n \times n$ matrix can be decomposed into two kernels of size $1 \times n$ and $n \times 1$. $1 \times n$ kernel produces an intermediate input which is then processed by a $n \times 1$ vector
    - Therefore, per output pixel per output channel, $n^2$ computations are reduced to $2n$
    - However, only a small fraction of all possible square kernels can be represented this way. This will reduce the representational power of the network by a lot.

    <img src="img/spatial-conv.png" width=500>


[Image source : Chi-Feng Wang](https://towardsdatascience.com/a-basic-introduction-to-separable-convolutions-b99ec3102728)


## Depthwise Separable Convolution

- [Francois Chollet](https://openaccess.thecvf.com/content_cvpr_2017/papers/Chollet_Xception_Deep_Learning_CVPR_2017_paper.pdf) proposed *depthwise convolution followed by pointwise convolution*


- *depthwise convolution*: It involves convoluting each input channel with an $n \times n$ kernel, thereby resulting in as many output channels as the number of input channels


- E.g., If the input image is of size $M \times M \times C_{in}$ then there will be $C_{in}$ number of $n \times n$ kernels that will be used to produce $C_{in}$ output channels. Assuming the output to be of same size, i.e., $M \times M$, there will be a total of $M \times M \times n \times n \times C_{in}$ multiplications involved. 


- *pointwise convolution*: To expand into $C_{out} > C_{in}$ convolutions, $C_{out}$ number of $1 \times 1$ convolution will be used. Note that the number of parameters will be $1 \times 1 \times C_{in}$ in each of these 1x1 convolution. There will be a total of $C_{out} \times M \times M \times C_{in}$ number of multiplications. 


- Thus, entire operation results in the following number of multiplication operations 
$$\text{Multiplications}_{DSC} = \Big(M \times M \times n \times n \times C_{in}\Big) + \Big( C_{out} \times M \times M \times C_{in}\Big)$$


- If a normal convolution has $C_{out}$ number of size $n$ kernels, there are a total of $n \times n \times C_{in}$ number of parameters for each kernel. For an output of size $M \times M \times C_{out}$, each pixel is computed by $n \times n \times C_{in}$ number of multiplications, thereby resulting in the following number of multiplication operations 

$$\text{Multiplications}_{normal} = M \times M \times C_{out} \times n \times n \times C_{in}$$ 


- Thus, total advantage of DSC can be seen in terms of computation reduction 

$$\frac{\text{Multiplications}_{DSC}}{\text{Multiplications}_{normal}} = \frac{1}{C_{out}} + \frac{1}{n^2}$$


<img src="img/dsc-1.png" width=500>
<img src="img/dsc-2.png" width=500>
<img src="img/dsc-3.png" width=500>


[[Francois Chollet 2017] Deep learning with Depthwise Separable Convolution](https://openaccess.thecvf.com/content_cvpr_2017/papers/Chollet_Xception_Deep_Learning_CVPR_2017_paper.pdf)

[Image source : Chi-Feng Wang](https://towardsdatascience.com/a-basic-introduction-to-separable-convolutions-b99ec3102728)

## Squeeze-and-excite: ResNet block



[[Hu et al. 2017] Squeeze-and-Excitation Networks](https://arxiv.org/abs/1709.01507)

## Neural Architecture Search (NAS)


- For a given dataset, searches for the best architecture constrained within a search space (i.e. number of filters, layers, etc.)

<img src="img/nas.png" width=1000>

[All publications on NAS](https://www.automl.org/automl/literature-on-neural-architecture-search/)
[[Elsken et al. 2018] Neural Architecture Search](https://www.ml4aad.org/wp-content/uploads/2018/09/chapter3-nas.pdf)

## Neural Architecture Search (NAS)


- **Search Space**: It defines the space of neural architectures to explore. Following search space is being considered in the community at the moment. 
    - Chain-structured nerual networks: number of layers, type of operations (e.g. convolution, pooling, etc.), hyperparameters, etc.
    - Multi-branch neural networks - design how previous layers should be connected,e.g, generalization of DenseNets
    - Cell design - Design of a block that can be stacked in a predefined manner
    - Meta-architecture search - design how different blocks should interact with each other


<table>
    <tr>
        <th style="text-align:center">Chain-structured or multi-branch network</th>
        <th style="text-align:center">Cell design / meta-architecuture search</th>
    </tr>
    <tr>
        <td><img src="img/chain-nas.png" width="500px"><br></td>
        <td><img src="img/cell-nas.png" width="500px"><br></td>
    </tr>
</table>

[All publications on NAS](https://www.automl.org/automl/literature-on-neural-architecture-search/)
[[Elsken et al. 2018] Neural Architecture Search](https://www.ml4aad.org/wp-content/uploads/2018/09/chapter3-nas.pdf)

## Neural Architecture Search (NAS)


- **Search strategy** can be designed using several techniques, e.g, 
    - Reinforcement Learning
    - Continuous Optimization
    - Evolutionary Algorithms
    - Bayesian Optimization
    

- **Performance Estimation Strategy**: Once the parameters are selected by a search strategy, this network needs to be trained on the dataset and evaluated on the validation dataset. This can take orders of GPU days. Following ideas have been explored in the community - 
    - Estimations based on *lower fidelities* of the actual performance after full training, e.g., "training times, training on a subset of data, or lower-resolution images, with less filters per layer"
    - *Learning curve extrapolation*, e.g., extrapolate initial learning curves and terminate those predicted to perform poorly, training a surrogate model to predict the performance, etc. 
    - *Network morphisms*: successive networks are initialized by the trained networks from the past to speedup training
    - *One-shot architecture search*: It treats all architectures as different subgraphs of a supergraph such that one one full network needs to be trained. All subnetworks can then be evaluated with little of no further training. 

[All publications on NAS](https://www.automl.org/automl/literature-on-neural-architecture-search/)
[[Elsken et al. 2018] Neural Architecture Search](https://www.ml4aad.org/wp-content/uploads/2018/09/chapter3-nas.pdf)

### Conclusions

Following factors have motivated the desig of new architectures:

- Learnability 
- More complex representations
- Decrease complexity of the network
- Increase generalization
- Reduce computational requirements and time to train

And the research still goes on to address the above factors ... 


Now open the following workbook `mna-practical.ipynb` to learn how to build an InceptionNet

<img src="img/jupyter.png" width="250px">
